In [1]:
import pandas as pd

df_rocket = pd.read_parquet("data/femnist/compressed/niid_rocket2d/testing.parquet")

df_vgg = pd.read_parquet("data/femnist/compressed/niid_vgg/testing.parquet")

In [4]:
df_rocket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83388 entries, 0 to 83387
Columns: 10002 entries, user to x_9999
dtypes: float32(10000), int64(1), object(1)
memory usage: 3.1+ GB


In [5]:
df_vgg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83388 entries, 0 to 83387
Columns: 4098 entries, user to x_4095
dtypes: float32(4096), int64(1), object(1)
memory usage: 1.3+ GB


In [1]:
import datasets.femnist as femnist
import os

type = "iid"

datasets = femnist.load(
        directory = os.path.join(
            'data', 
            'femnist', 
            'compressed', 
            type
        )
    )

centralized = femnist.centralized(datasets)

In [2]:
from torch.utils.data import DataLoader
from r2d2 import R2D2
import torch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bs = 1024
centralized_dl = DataLoader(centralized['training'], batch_size=bs, shuffle=True)

print(len(centralized_dl))

transform = R2D2(
    h = 28,
    w = 28,
    ch = 1
).to(device=device)

print("beginnin transform")
tensor_list = []
count = 0
for xb, yb in centralized_dl:
    if count >= 10:
        break
    xb = xb.to(device=device)
    tensor_list.append(transform(xb))
    count += 1

print("beginning pca")
pca = PCA(0.99999)
scaler = StandardScaler()

scaler.fit(pca.fit_transform(torch.cat(tensor_list).numpy()))
pca.n_components_

576
beginnin transform
beginning pca


8447

In [3]:
from torch.utils.data import TensorDataset, Dataset
from tqdm import tqdm

def transform_dataset(dataset: dict[str, list[tuple[str, Dataset]]], r2d2: torch.nn.Module, pca: PCA, scaler: StandardScaler):
    for k in dataset:
        data = dataset[k]
        new_data = []
        progress_bar = tqdm(total=len(data), desc=f"{k}")
        for id, samples in data:
            list_x = []
            list_y = []
            for x, y in DataLoader(samples, batch_size=2048):
                x = r2d2(x.to(device=device))
                x = x.numpy()
                x = scaler.transform(pca.transform(x))
                x = torch.tensor(x)
                list_x.append(x)
                list_y.append(y)
            transformed_samples = TensorDataset(torch.cat(list_x), torch.cat(list_y))
            new_data.append((id, transformed_samples))
            progress_bar.update(1)
        progress_bar.close()
        dataset[k] = new_data

In [4]:
transform_dataset(dataset=datasets, r2d2=transform, pca=pca, scaler=scaler)

training:   1%|          | 23/2877 [02:39<5:15:21,  6.63s/it]

KeyboardInterrupt: 